In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import torch
import pickle
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchtext.data.functional import to_map_style_dataset
from torch import nn
from itertools import chain
from collections import Counter

In [2]:
# Load data
pickle_in = open("data_withMeasurements.pkl", "rb")
dataWIKI = pickle.load(pickle_in)

# Load data
pickle_in = open("dataBOW_withMeasurements.pkl", "rb")
dataBOW = pickle.load(pickle_in)

In [4]:
# Add data together
data = dataWIKI | dataBOW
#data = dataWIKI
#data = dataBOW

In [5]:
TotalValues = list(chain.from_iterable(data.values()))

ones = Counter(ones[0] for ones in TotalValues if ones[0] == 1)
zeros = Counter(ones[0] for ones in TotalValues if ones[0] == 0)

print('{0} values.'. format(len(TotalValues)))
print('{0} labels with 1 (true).'.format(ones[1]))
print('{0} labels with 0 (false).'.format(zeros[0]))

47997 values.
9639 labels with 1 (true).
38358 labels with 0 (false).


In [ ]:
'''
# Species
len(data)
# Total values
Total = sum(len(v) for v in list(data.values()))
print(Total)
# Total Truths
TotalValues = [v for v in list(data.values())]
Truths = [[ones[0] for ones in Values if ones[0] == 1] for Values in TotalValues]
AmountofTruths = sum(len(x) for x in Truths)
# Total False
AmountofFalse = Total - AmountofTruths
print(AmountofTruths)
print(AmountofFalse)
'''

In [6]:
# Extract values from the dict
data = list(chain.from_iterable(data.values()))
# Train test sequence
train = int(len(data) * 0.8)
test = len(data) - train

In [7]:
# Split the dataset
trainset = data[0:train]
testset = data[train:]

In [8]:
len(trainset)

38397

In [9]:
# Basic English
tokenizer = get_tokenizer('basic_english')

# Tokenize the dataset
def yield_tokens(data_iter):
    # Drop the label (label, text)
    for _, text in data_iter:
        yield tokenizer(text)

# Build a vocabulary        
vocab = build_vocab_from_iterator(yield_tokens(trainset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [10]:
# Create a pipeline
text_pipeline = lambda x: vocab(tokenizer(x))
# -1 could be removed if data is loaded differently
label_pipeline = lambda x: int(x) 

In [11]:
# Testing
proc = torch.tensor(text_pipeline('the a bird birds, the bear are not equal.'), dtype=torch.int64)
print(proc)

tensor([   1,    8,  441,  114,    2,    1,  318,    9,   37, 2708,    6])


In [12]:
# Set device (CPU for macs)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    
    '''
    Convert sentences or paragrahs to integers by using
    the PyTorch Vocab(). The data is converted and 
    returned as a tensor. The offset of the words is 
    compared to the start of the sentence/paragraph.
    '''

    # Init lists
    label_list, text_list, offsets = [], [], [0]
    
    # Loop over the data
    for (_label, _text) in batch:
        # Append the labels to list
        label_list.append(label_pipeline(_label))
        # Process the text (singed 64), and convert to tensor
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        # Append the text to list 
        text_list.append(processed_text)
        # Append the offset (tensor size)   
        offsets.append(processed_text.size(0))
    
    # Convert the label list to a tensor
    label_list = torch.tensor(label_list, dtype=torch.int64)
    # Cummulative sum the offsets (dim=0 == rowwise)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    # Concatenate the text list
    text_list = torch.cat(text_list)
    
    # Return the values
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [13]:
train_iter = trainset
dataloader = DataLoader(train_iter, batch_size=16, shuffle=True, collate_fn=collate_batch)

In [14]:
next(iter(dataloader))

(tensor([0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]),
 tensor([   14,     1,  1305,  5478,    41,     2,    11,   634,    97,    43,
            17,   541,    68,  1344,     3,  4489,    16,     8,  2197,     5,
            46,  1488,     3,  2894,     1,   132,  8285,     5,     1,   543,
             9,   100,  8777,    24,   383,   973,     3,   316,   162,     2,
            31,    75,     9,    70,   841,    11,    22,   150,     4,   247,
             6,     1,   631,     5,     1,   173,   111,   258,  1177,    10,
          5766,   900,     3,  1459,     7,  1217,  5924,     1,  1131,   847,
            17,  1573,   868,    44,   171,    51,     7,     1,    78,     6,
             1,    53,    29,   949,   290,    10,  6486,    36,  1060,     9,
          2740,     5,   195,    16,  3174,   902,     3,  7884,    30,   350,
             5,  2121,    10,    26,   435,  2319,     4,    14, 13808,    52,
             6,   690,    11,     9,     1,    53,  1527,   389,     5, 

In [15]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [16]:
train_iter = trainset
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [17]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [18]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 8 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = trainset, testset
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 4560 batches | accuracy    0.840
| epoch   1 |  1000/ 4560 batches | accuracy    0.875
| epoch   1 |  1500/ 4560 batches | accuracy    0.904
| epoch   1 |  2000/ 4560 batches | accuracy    0.893
| epoch   1 |  2500/ 4560 batches | accuracy    0.905
| epoch   1 |  3000/ 4560 batches | accuracy    0.903
| epoch   1 |  3500/ 4560 batches | accuracy    0.906
| epoch   1 |  4000/ 4560 batches | accuracy    0.921
| epoch   1 |  4500/ 4560 batches | accuracy    0.912
-----------------------------------------------------------
| end of epoch   1 | time:  4.80s | valid accuracy    0.918 
-----------------------------------------------------------
| epoch   2 |   500/ 4560 batches | accuracy    0.926
| epoch   2 |  1000/ 4560 batches | accuracy    0.919
| epoch   2 |  1500/ 4560 batches | accuracy    0.926
| epoch   2 |  2000/ 4560 batches | accuracy    0.932
| epoch   2 |  2500/ 4560 batches | accuracy    0.929
| epoch   2 |  3000/ 4560 batches | accuracy    0.926
| epoch  

In [19]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.855


In [ ]:
label = {1: "a description or similar.",
         0: "something else."}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 

# Measurements
ex_text_str = 'They are 90 to 150 cm tall at the shoulder and can tower at an intimidating height of 8 feet when standing upright on their hind legs.'
model = model.to("cpu")
print("This is %s" %label[predict(ex_text_str, text_pipeline)])

# Random
ex_text_str = 'Hi I am GIS student, this is a random sentence!'
model = model.to("cpu")
print("This is %s" %label[predict(ex_text_str, text_pipeline)])

# Bird stuff
ex_text_str = 'The bill is long and orange.'
model = model.to("cpu")
print("This is %s" %label[predict(ex_text_str, text_pipeline)])

# Something about bears
ex_text_str = '''Brown bears are often not fully brown. 
                They have long, thick fur, with a moderately long mane at the back of the neck which varies somewhat across the types. 
                In India, brown bears can be reddish with silver-tipped hairs, while in China brown bears are bicolored, 
                with a yellowish-brown or whitish collar across the neck, chest and shoulders.'''

model = model.to("cpu")
print("This is %s" %label[predict(ex_text_str, text_pipeline)])

# Somehting about Robins
ex_text_str = '''The upperparts are brownish, or olive-tinged in British birds, and the belly whitish, while the legs and feet are brown. 
                The bill and eyes are black.'''

model = model.to("cpu")
print("This is %s" %label[predict(ex_text_str, text_pipeline)])

# Random Difficult sentence
ex_text_str = '''While I am very tan from the sun, I can be pale within a few days.
                I have blonde hair'''

model = model.to("cpu")
print("This is %s" %label[predict(ex_text_str, text_pipeline)])

In [ ]:
# init dicts
ExtraTesting = []

# Removes references in text
ReferenceRemover = '\[\d*\]'

URL = 'https://en.wikipedia.org/wiki/Eurasian_wigeon'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
    
for Tags in soup.find_all('h2'):

    # Skip useless/empty stuff
    if Tags.span == None:
        continue

    # Set chapter variable    
    Chapter = Tags.span.attrs['id']

    # Check if the chapter is description (or similar)
    if Chapter == 'Characteristics'or \
       Chapter == 'Description' or \
       Chapter == 'Appearance':

        # Get the next sibling (text)
        for Text in Tags.find_next_siblings('p'):

            # Add description data to dict
            if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                # Remove source
                Paragraph = re.sub(ReferenceRemover, '', Text.text)
                # Split into Sentences
                SentenceList = Paragraph.split('. ')
                # Add to the dict
                ExtraTesting += [(1, Sentence) for Sentence in SentenceList]

            # Add non description data to dict
            elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                # Remove source
                Paragraph = re.sub(ReferenceRemover, '', Text.text)
                # Split into Sentences
                SentenceList = Paragraph.split('. ')
                # Add to the dict
                ExtraTesting += [(0, Sentence) for Sentence in SentenceList]

In [ ]:
label = {1: "a description or similar.",
         0: "something else."}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 

for tests in ExtraTesting:
    ex_text_str = tests[1]
    
    model = model.to("cpu")

    print("This is %s" %label[predict(ex_text_str, text_pipeline)])
    print("Real value was {0}".format(tests[0]))
    print(tests[1])
    print('\n')